In [1]:
%%writefile requirements.txt
chromadb
langchain
langchain_community
sentence_transformers

Overwriting requirements.txt


## Required Modules

In [ ]:
from tqdm.autonotebook import tqdm, trange

In [2]:
import os
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_transformers import (
    EmbeddingsRedundantFilter,
    EmbeddingsClusteringFilter,
)
# from langchain.retrievers.document_compressors import DocumentCompressorPipeline
# from langchain.retrievers import ContextualCompressionRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from langchain.docstore.document import Document

# Embedding Model

In [3]:
import time
s = time.time()
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
print("Embedding Model Loaded in", time.time() - s, "seconds")

C:\Users\VISHAL MAURYA\Desktop\Document Comparision\pdr_venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding Model Loaded in 89.81707167625427 seconds


## Data Pre-processing

In [4]:
# pdf_paths = ["data/GB 25991-2010 Automotive Headlamps with LED Light Sources andor LED Modules (1) (2).pdf"]
pdf_paths = ["Chinese Documents/GB 4785-2019 Lighting installation -EN.pdf"]

In [5]:
def pdf_paths_to_documents(pdf_paths):
    documents = []
    for path in pdf_paths:
        loader = PyMuPDFLoader(path)
        list_of_documents = loader.load()
        combined_text = "\n".join(document.page_content for document in list_of_documents)
        document = Document(page_content=combined_text, metadata={"source": path})
        documents.append(document)
    return documents

In [6]:
documents = pdf_paths_to_documents(pdf_paths)

In [7]:
documents

[Document(page_content=" \n \n  \n \n \n \n \n \n \n \n \n \n \nJanuary / 2020 \n \nACEA Translation  \n1 \nFor Reference Purposes Only \nICS 43.040.20 \nT 38 \n \n \nNational Standard of the People’s Republic of China \n \nGB 4785-2019 \nReplace GB 4785-2007 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n \n \n \n \n \n \nProvisions for installation of the \nexternal lighting and light-signalling \ndevices for motor vehicles and their \ntrailers \n \n \n \n \n \n \n \n \n \n \n \n \nPublishing date: 2019-12-17    \n \n \n \n \n Implementation date: 2020-07-01 \n \nPromulgated by \nState Administration for Market Regulation \nAnd \nStandardization Administration of the People's Republic of China \n\n \n \n  \n \n \n \n \n \n \n \n \n \n \nJanuary / 2020 \n \nACEA Translation  \n2 \nFor Reference Purposes Only \nForeword \nAll technical content of this Standard are mandatory. \nThis Standard is drafted in according with the rules given in the GB/T 1.1-2009. \nThis 

## Parent Document Retriever
Version 1 - Smaller chunks

In [8]:
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever

In [9]:
def documents_to_retriever(documents):
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=100)
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
    
    vectorstore = Chroma(collection_name="full_documents", 
                         embedding_function=hf)
    store = InMemoryStore()
    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter)
    
    s = time.time()
    retriever.add_documents(documents, ids=None)
    print("Document splitted in", time.time()-s, "seconds")

    return retriever

In [10]:
retriever = documents_to_retriever(documents)

Document splitted in 1797.8550481796265 seconds


Above is time taken in child chunking, parent chunking and creation of vectorstore

In [1]:
query = "The color of the light emitted by LED headlamps shall be?"  
# sub_docs = vectorstore.similarity_search(query)
# sub_docs

In [13]:
retrieved_docs = retriever.get_relevant_documents(query)
retrieved_docs

C:\Users\VISHAL MAURYA\Desktop\Document Comparision\pdr_venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='It shall remain activated while the failure is present. It may be cancelled \ntemporarily, but shall be repeated whenever the device, which starts and stops \nthe engine, is switched on and off. \n5.2.9 \nOther requirements \nThe requirements of 4.5.b of this Standard shall not apply to dipped-beam \nheadlamps. \nUnilateral dipped-beam headlamps with a light source or LED module(s) \nproducing the principal dipped-beam and having a total luminous flux which \nexceeds 2,000 lm, shall be installed with a headlamp cleaner in compliance with \nGB 21260.', metadata={'source': 'Chinese Documents/GB 4785-2019 Lighting installation -EN.pdf'}),
 Document(page_content='January / 2020 \n \nACEA Translation  \n40 \nFor Reference Purposes Only \nremain switched on during the main-beam operation. \n5.2.7.4 \nOne additional light source or one or more LED module(s), located inside the \ndipped-beam headlamps or in a lamp (except the main-beam headlamp) \ngrouped or reciprocall

In [14]:
def query_to_context(query, retriever):
    context = ""
    retrieved_docs = retriever.get_relevant_documents(query)
    for i in range(len(retrieved_docs)):
      # print("----------------------------------------- parent",i+1)
        # print(retrieved_docs[i].page_content)
        context += retrieved_docs[i].page_content
    return context

In [15]:
query1 = "The LED module of headlamp shall be operated for how many hours before cooled down toambient temperature before starting the tests?"
context = query_to_context(query1, retriever)
context

'headlamp shall be switched off. \nManufacturer shall demonstrate conformity of product with above clauses to \ninspection institution by using simulation or other method. If necessary, use \nilluminance meter featuring cosine correction and measure horizontal \nilluminance at height identical with vehicle sensor. It is allowed to prove \nconformity with this clause by using method approved by inspection institution or \ndocument or other method. \n5.22.9.4 \nFor all light ing units that are switched on simultaneously in main-beam light ingJanuary / 2020 \n \nACEA Translation  \n119 \nFor Reference Purposes Only \n \nAnnex H \n(Normative) \nAutomatic Switching of Dipped-beam Headlamps \nConditions of automatic switching of dipped-beam headlamps are shown in Table H.1 \nTable H1 \nConditions of automatic switching of dipped-beam headlamps \nAmbient light outside the vehicle Note \nDipped-beam headlamps \nResponse time \nLess than 1,000lux \nON \nNot more than 2s \n1,000lux to 7,000lux \

In [16]:
query2 = "Tell me about Test of stability of photometric performance."
context = query_to_context(query2, retriever)
context

'January / 2020 \n \nACEA Translation  \n42 \nFor Reference Purposes Only \nThe above-mentioned “objective luminous flux” is subject to test voltage. \nFor filament bulb allowing usage of multiple test voltages, the above-mentioned \n“objective luminous flux” producing principal dipped-beam is subject to product \ntest voltage used for type test of light sources. \nOnly the dipped-beam lamps in compliance with GB 4599 or GB 21259 or GB \n25991 may be used for realizing the bend lighting. \nFor a left turn, if bend lighting is produced by a horizontal movement of thelamps complying with 5.18.7 below, and the emergency stop signal may be \nflashing lamps. \nb)The photometric characteristics of any lamp may vary: \n-- \nIn relation to the ambient light; \n-- \nAs a consequence of the activation of other lamps; or \n-- \nWhen the lamps is being used to provide another lighting function. \nProvided that any variation in the photometric characteristics above is in \ncompliance with the relat

In [17]:
query3 = "What are environmental conditions for test for change in vertical position of the cut-off line under the influence ofheat?"
context = query_to_context(query3, retriever)
context

"the front fog beam in relation to the prevailing ambient conditions, provided that \nthe limits for the vertical inclination specified in 5.3.6.1.2.b.1 are not exceeded. \n5.3.6.2.4 \nIn the case of a failure of the levelling device, the front fog beam shall not \nassume a position in which the cut off is less inclined than it was at the time \nwhen the failure of the device occurred. \n5.3.6.2.5 \nIf the vehicle structure can ensure that under all loading conditions specified in \nAnnex A, can meet the requirements of 5.3.6.1.2.b.1, allowing not install levelingof 0.1% and one to one correspondence. \n5.22.6.1.2 \nFor horizontal part of cutoff line of basic dipped-beam, it is necessary to ensure \nthat under all loading conditions specified in Annex A, its downward inclination \nshall be within relevant limit scope in 5.2.6.1.2 of this regulation, including that \ninitial aiming shall meet relevant requirements. \nWhere dipped-beam is constituted by different beams of different light

In [18]:
query4 = "For headlamp with the outside lens in glass, a mixture to be applied to the headlamp lens shall consist of which elements"
context = query_to_context(query4, retriever)
context

'Note: lamps with illuminating surface of a lighting device include: Main-beam headlamp, \ndipped-beam headlamp, front fog lamp, reversing lamp, cornering lamp and AFS. \n3.11.2 \nIlluminating surface of a light-signalling device other than a retro-reflector  \nThe orthogonal projection of the lamp in a plane perpendicular to its axis of \nreference and in contact with the exterior light emitting surface of the lamp, thisIt shall remain activated while the failure is present. It may be cancelled \ntemporarily, but shall be repeated whenever the device, which starts and stops \nthe engine, is switched on and off. \n5.2.9 \nOther requirements \nThe requirements of 4.5.b of this Standard shall not apply to dipped-beam \nheadlamps. \nUnilateral dipped-beam headlamps with a light source or LED module(s) \nproducing the principal dipped-beam and having a total luminous flux which \nexceeds 2,000 lm, shall be installed with a headlamp cleaner in compliance with \nGB 21260.January / 2020 \n \n

## Generation

In [19]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
import time

In [20]:
model = Ollama(model="llama2")
# embeddings = OllamaEmbeddings(model="llama2")

In [21]:
from langchain.prompts import PromptTemplate

def query_to_prompt(query):
    context = query_to_context(query, retriever)
    template = """
    Answer the question based on the context below. 
    Don't start the response from "Based on the context provided, the answer to the question is:" or something like that, just given the answer.
    If you can't answer the question, reply "I don't know". 
    This is very important for my life, be serious and precise.
    
    Context: {context}
    
    Question: {question}
    """
    prompttemplate = PromptTemplate.from_template(template)
    prompt = prompttemplate.format(context = context, question = query)
    return prompt

In [22]:
# def query_to_prompt(query):
#     context = query_to_context(query)
#     prompt = f"Answer the question based on the context below. Don't start the response from 'Based on the context provided, the answer to the question is:' or something like that, just given the answer.If you can't answer the question, reply 'I don't know'. 
#     This is very important for my life, be serious and precise. CONTEXT: {context} QUESTION: {query}"
#     return prompt

In [25]:
queries = [query, query1, query2, query3, query4] # For Automotive Headlamps

In [ ]:
for query in queries:
    st = time.time()
    print("QUERY:", query)
    prompt = query_to_prompt(query)
    print("RESPONSE:", model.invoke(prompt))
    print("GENERATION TIME:", (time.time() - st)/60, "minutes")
    print("")

In [23]:
new_queries = ["Whats the difference between Grouped and Combined lamps?", "Can  dipped-beam headlamp and main-beam headlamp for front lighting system?", "what is color of End Outline marker lamp?", "Can yellow lamp used as front fog lamp?", "Can red color light placed in the front of the vechiile?", "Can white light can be placed at the back of the vechile?", "What are 1,1,a,1b,2a,2b,5,6 in direction indicator lamps?", "is cornering lamp mandatory?", "does reflective tape come under light and light signalling?", "standard weight of a person for testing?","can dipped beam uses as a main beam?", "what are the light functions to be kept rear of the vechile?"]

In [24]:
for query in new_queries:
    st = time.time()
    print("QUERY:", query)
    prompt = query_to_prompt(query)
    print("RESPONSE:", model.invoke(prompt))
    print("GENERATION TIME:", (time.time() - st)/60, "minutes")
    print("")

QUERY: Whats the difference between Grouped and Combined lamps?
RESPONSE: Grouped and combined lamps differ in their design and purpose. Here's the difference between them based on the given context:

Grouped lamps:

* Have separate apparent surfaces but share a common lamp body.
* Are positioned in the same direction as the reference axis.
* Do not overlap based on the areas bounded by the outline of their light emitting surfaces.

Combined lamps:

* Have separate apparent surfaces but share a common light source and lamp body.
* Are positioned in the same direction as the reference axis.
* Do not overlap based on the areas bounded by the outline of their light emitting surfaces, unlike grouped lamps.

In summary, grouped lamps are designed to provide multiple light sources from different parts of the same lamp body, while combined lamps share a common light source and lamp body but have separate apparent surfaces.
GENERATION TIME: 8.348908106486002 minutes

QUERY: Can  dipped-beam he

### 1

In [ ]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
# model_name = "EleutherAI/gpt-neo-2.7B"

In [ ]:
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
# model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
# input_text = "Your input text here"
# input_ids = tokenizer.encode(input_text, return_tensors="pt")

# # Generate response
# output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2)
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(generated_text)


# Reranking

In [ ]:
pip install cohere

In [ ]:
import cohere
co = cohere.Client('w8CnnlzVol2aZEiirZNLUs0onAqXUUYBZCw2Oj7g')

In [ ]:
retrieved_docs_query1=[]
for i in range(len(retrieved_docs_1)):
  retrieved_docs_query1.append(retrieved_docs_1[i].page_content)

In [ ]:
results = co.rerank(query=query_1, documents=retrieved_docs_query1, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.

In [ ]:
results

In [ ]:
print(query_1)

In [ ]:
results = co.rerank(query=query_1, documents=retrieved_docs_query1, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
    print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
    print(f"Source Document: {retrieved_docs_1[r.index].metadata}")
    print(f"Document: {retrieved_docs_query1[r.index]}")
    print(f"Relevance Score: {r.relevance_score:.2f}")
    print("\n")

In [ ]:
retrieved_docs_query2=[]
for i in range(len(retrieved_docs_2)):
  retrieved_docs_query2.append(retrieved_docs_2[i].page_content)

In [ ]:
print(query_2)

In [ ]:
results = co.rerank(query=query_2, documents=retrieved_docs_query2, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_2[r.index].metadata}")
  print(f"Document: {retrieved_docs_query2[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

In [ ]:
retrieved_docs_query3=[]
for i in range(len(retrieved_docs_3)):
  retrieved_docs_query3.append(retrieved_docs_3[i].page_content)

print(query_3,"\n")
results = co.rerank(query=query_3, documents=retrieved_docs_query3, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_3[r.index].metadata}")
  print(f"Document: {retrieved_docs_query3[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

In [ ]:
retrieved_docs_query4=[]
for i in range(len(retrieved_docs_4)):
  retrieved_docs_query4.append(retrieved_docs_4[i].page_content)

print(query_4,"\n")
results = co.rerank(query=query_4, documents=retrieved_docs_query4, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_4[r.index].metadata}")
  print(f"Document: {retrieved_docs_query4[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

In [ ]:
retrieved_docs_query5=[]
for i in range(len(retrieved_docs_5)):
  retrieved_docs_query5.append(retrieved_docs_5[i].page_content)

print(query_5,"\n")
results = co.rerank(query=query_5, documents=retrieved_docs_query5, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_5[r.index].metadata}")
  print(f"Document: {retrieved_docs_query5[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

# Parent Document Retriever
Version 2 - Larger chunks

In [ ]:
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever

child_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=hf
)
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

In [ ]:
# adding original documents before splitting or chunking as the retriever already does these
retriever.add_documents(documents, ids=None)

In [ ]:
list(store.yield_keys())

## Query 1

In [ ]:
query_1=" What are some challenges associated with data collection?"
sub_docs_1 = vectorstore.similarity_search(query_1)

In [ ]:
print(sub_docs_1[0].page_content)

In [ ]:
sub_docs_1

In [ ]:
retrieved_docs_1 = retriever.get_relevant_documents(query_1)

In [ ]:
len(retrieved_docs_1[0].page_content)

In [ ]:
retrieved_docs_1

## Query 2

In [ ]:
#query_1=" What are some challenges associated with data collection?"
query_2=" What are the advantages of the Apriori algorithm?"

sub_docs_2 = vectorstore.similarity_search(query_2)
retrieved_docs_2 = retriever.get_relevant_documents(query_2)
retrieved_docs_2

In [ ]:
sub_docs_2

## Query 3

In [ ]:
#query_1=" What are some challenges associated with data collection?"
#query_2=" What are the advantages of the Apriori algorithm?"
query_3=" What distinguishes training data in supervised learning, and what is its purpose? "

sub_docs_3 = vectorstore.similarity_search(query_3)
retrieved_docs_3 = retriever.get_relevant_documents(query_3)
retrieved_docs_3

## Query 4

In [ ]:
#query_1=" What are some challenges associated with data collection?"
#query_2=" What are the advantages of the Apriori algorithm?"
#query_3=" What distinguishes training data in supervised learning, and what is its purpose? "
query_4=" What is cross-validation?"

sub_docs_4 = vectorstore.similarity_search(query_4)
retrieved_docs_4 = retriever.get_relevant_documents(query_4)
retrieved_docs_4

In [ ]:
sub_docs_4

## Query 5

In [ ]:
#query_1=" What are some challenges associated with data collection?"
#query_2=" What are the advantages of the Apriori algorithm?"
#query_3=" What distinguishes training data in supervised learning, and what is its purpose? "
#query_4=" What is cross-validation?"
query_5="What is the process of building the decision tree classifier, and how is it trained on the dataset?"

sub_docs_5 = vectorstore.similarity_search(query_5)
retrieved_docs_5 = retriever.get_relevant_documents(query_5)
retrieved_docs_5

In [ ]:
sub_docs_5

## Query 6

In [ ]:
query_6="What is the challenge associated with supervised learning?"

sub_docs_6 = vectorstore.similarity_search(query_6)
retrieved_docs_6 = retriever.get_relevant_documents(query_6)
retrieved_docs_6

## Query 7

In [ ]:
query_7="What considerations should businesses keep in mind when applying market basket analysis to understand customer behavior?"

sub_docs_7 = vectorstore.similarity_search(query_7)
retrieved_docs_7 = retriever.get_relevant_documents(query_7)
retrieved_docs_7

## Query 8

In [ ]:
query_8="How does the genie index contribute to decision tree construction, and why is it favored in practice, particularly in Python libraries?"

sub_docs_8 = vectorstore.similarity_search(query_8)
retrieved_docs_8 = retriever.get_relevant_documents(query_8)
retrieved_docs_8

## Query 9

In [ ]:
query_9="What is the harmonic mean, and how does it enable the combination of precision and recall into a single metric?"

sub_docs_9 = vectorstore.similarity_search(query_9)
retrieved_docs_9 = retriever.get_relevant_documents(query_9)
retrieved_docs_9

## Query 10

In [ ]:
query_10="What are the implications of choosing a large or small learning rate?"

sub_docs_10 = vectorstore.similarity_search(query_10)
retrieved_docs_10 = retriever.get_relevant_documents(query_10)
retrieved_docs_10

In [ ]:
pip install cohere

# Co-here Re-ranking

In [ ]:
import cohere
co = cohere.Client('w8CnnlzVol2aZEiirZNLUs0onAqXUUYBZCw2Oj7g')

## Query 1

In [ ]:
retrieved_docs_query1=[]
for i in range(len(retrieved_docs_1)):
  retrieved_docs_query1.append(retrieved_docs_1[i].page_content)

In [ ]:
results = co.rerank(query=query_1, documents=retrieved_docs_query1, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.

In [ ]:
results

In [ ]:
print(query_1)

In [ ]:
#results = co.rerank(query=query_1, documents=retrieved_docs_query1, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
print(query_1,"\n")
for idx, r in enumerate(results.results):
    print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
    print(f"Source Document: {retrieved_docs_1[r.index].metadata}")
    print(f"Document: {retrieved_docs_query1[r.index]}")
    print(f"Relevance Score: {r.relevance_score:.2f}")
    print("\n")

## Query 2

In [ ]:
retrieved_docs_query2=[]
for i in range(len(retrieved_docs_2)):
  retrieved_docs_query2.append(retrieved_docs_2[i].page_content)

In [ ]:
print(query_2)

In [ ]:
results = co.rerank(query=query_2, documents=retrieved_docs_query2, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
print(query_2,"\n")
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_2[r.index].metadata}")
  print(f"Document: {retrieved_docs_query2[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## Query 3

In [ ]:
retrieved_docs_query3=[]
for i in range(len(retrieved_docs_3)):
  retrieved_docs_query3.append(retrieved_docs_3[i].page_content)

print(query_3,"\n")
results = co.rerank(query=query_3, documents=retrieved_docs_query3, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_3[r.index].metadata}")
  print(f"Document: {retrieved_docs_query3[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## Query 4

In [ ]:
retrieved_docs_query4=[]
for i in range(len(retrieved_docs_4)):
  retrieved_docs_query4.append(retrieved_docs_4[i].page_content)

print(query_4,"\n")
results = co.rerank(query=query_4, documents=retrieved_docs_query4, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_4[r.index].metadata}")
  print(f"Document: {retrieved_docs_query4[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## Query 5

In [ ]:
retrieved_docs_query5=[]
for i in range(len(retrieved_docs_5)):
  retrieved_docs_query5.append(retrieved_docs_5[i].page_content)

print(query_5,"\n")
results = co.rerank(query=query_5, documents=retrieved_docs_query5, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_5[r.index].metadata}")
  print(f"Document: {retrieved_docs_query5[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## Query 6

In [ ]:
retrieved_docs_query6=[]
for i in range(len(retrieved_docs_6)):
  retrieved_docs_query6.append(retrieved_docs_6[i].page_content)

print(query_6,"\n")
results = co.rerank(query=query_6, documents=retrieved_docs_query6, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_6[r.index].metadata}")
  print(f"Document: {retrieved_docs_query6[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## Query 7

In [ ]:
retrieved_docs_query7=[]
for i in range(len(retrieved_docs_7)):
  retrieved_docs_query7.append(retrieved_docs_7[i].page_content)

print(query_7,"\n")
results = co.rerank(query=query_7, documents=retrieved_docs_query7, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_7[r.index].metadata}")
  print(f"Document: {retrieved_docs_query7[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## Query 8

In [ ]:
retrieved_docs_query8=[]
for i in range(len(retrieved_docs_8)):
  retrieved_docs_query8.append(retrieved_docs_8[i].page_content)

print(query_8,"\n")
results = co.rerank(query=query_8, documents=retrieved_docs_query8, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_8[r.index].metadata}")
  print(f"Document: {retrieved_docs_query8[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## Query 9

In [ ]:
retrieved_docs_query9=[]
for i in range(len(retrieved_docs_9)):
  retrieved_docs_query9.append(retrieved_docs_9[i].page_content)

print(query_9,"\n")
results = co.rerank(query=query_9, documents=retrieved_docs_query9, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_9[r.index].metadata}")
  print(f"Document: {retrieved_docs_query9[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## Query 10

In [ ]:
retrieved_docs_query10=[]
for i in range(len(retrieved_docs_10)):
  retrieved_docs_query10.append(retrieved_docs_10[i].page_content)

print(query_10,"\n")
results = co.rerank(query=query_10, documents=retrieved_docs_query10, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Source Document: {retrieved_docs_10[r.index].metadata}")
  print(f"Document: {retrieved_docs_query10[r.index]}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

In [ ]:
# list(store.yield_keys())

In [ ]:
# from langchain_community.document_loaders import TextLoader
# from langchain_community.document_loaders import DirectoryLoader
# # Load and process the text files
# loader = DirectoryLoader('data/', glob="./*.txt", loader_cls=TextLoader)
# documents = loader.load()  # text file to [document]

# loader = PyPDFLoader(path)
# documents = loader.load()
# len(documents)  # List of documents for each page